マウント

In [24]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [25]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# check auth
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

データセットの読み込み

In [33]:
# downloaded = drive.CreateFile({'id':'https://drive.google.com/file/d/16Ust5fz2MqpQOEXVxUtnGF2gktEzwmqO/view?usp=sharing'})

# # Download the file to a local disk as 'sample.csv'.
# downloaded.GetContentFile('/content/drive/My Drive/Colab Notebooks/Workschool/titanic_assignment_question.csv')

import pandas as pd
data_pred = pd.read_csv('drive/My Drive/Colab Notebooks/Workschool/titanic_assignment_question.csv')
data_pred.head()

,PassengerId,Sex,Pclass,Fare
0,2,1,1,71.2833
1,6,0,3,8.4583
2,9,1,3,11.1333
3,15,1,3,7.8542
4,28,0,1,263.0000


In [34]:
titanic = pd.read_csv('drive/My Drive/Colab Notebooks/Workschool/titanic_text.csv')
titanic.head()

,PassengerId,Survived,Sex,Pclass,Fare
0,1,0,0,3,7.2500
1,3,1,1,3,7.9250
2,4,1,1,1,53.1000
3,5,0,0,3,8.0500
4,7,0,0,1,51.8625


In [35]:
import numpy as np
import pandas as pd
import sklearn

In [36]:
print(titanic.columns)
y = titanic["Survived"]
x = np.array(titanic.drop(["Survived", "PassengerId"], axis=1))

# numpy配列に変換
y_array = np.array(y)
x_array = np.array(x)

print("y_array: ", y_array.shape)
print("x_array: ", x_array.shape)

Index(['PassengerId', 'Survived', 'Sex', 'Pclass', 'Fare'], dtype='object')
y_array:  (712,)
x_array:  (712, 3)


データを訓練データとテストデータに分ける

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
x_train, x_test, y_train, y_test = train_test_split(x_array, y_array, test_size=0.4, random_state=0)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_test: ", x_test.shape)
print("y_test: ", y_test.shape)

x_train:  (427, 3)
y_train:  (427,)
x_test:  (285, 3)
y_test:  (285,)


モデルの作成

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
rfc = RandomForestClassifier(random_state=0)

rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [41]:
y_pred = rfc.predict(x_test)

In [42]:
# 精度を測る
from sklearn.metrics import accuracy_score
accuracy_score(y_pred, y_test)

0.8

# 用意されたデータに対して作成したモデルで予測結果を出す

In [54]:
x_test = np.array(data_pred.drop(["PassengerId"], axis=1))

y_pred = rfc.predict(x_test)
print(y_pred)
print(type(y_pred))
print(y_pred.shape)

[1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 1 0 1 0 1 0
 1 1 0 0 1 1 0 0 0 1 1 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 0 0 1 1 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0
 0 1 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0]
<class 'numpy.ndarray'>
(179,)


PassengerId と Survived を並べたDataFrameをCSVとして書き出す

In [61]:
data_passengerId = data_pred["PassengerId"]
print(data_passengerId.shape)
print(data_passengerId)
data_survived = pd.Series(y_pred, name="Survived")
print(data_survived)

y_pred_assignment_submit = pd.concat([data_passengerId, data_survived], axis=1)
print(y_pred_assignment_submit)

y_pred_assignment_submit.to_csv("y_pred_assignment_submit.csv", index=None)

(179,)
0        2
1        6
2        9
3       15
4       28
      ... 
174    878
175    881
176    884
177    888
178    891
Name: PassengerId, Length: 179, dtype: int64
0      1
1      0
2      1
3      1
4      0
      ..
174    0
175    1
176    0
177    1
178    0
Name: Survived, Length: 179, dtype: int64
     PassengerId  Survived
0              2         1
1              6         0
2              9         1
3             15         1
4             28         0
..           ...       ...
174          878         0
175          881         1
176          884         0
177          888         1
178          891         0

[179 rows x 2 columns]
